## Day 12: Passage Pathing
[link](https://adventofcode.com/2021/day/12)

In [ ]:
#!value --name sampleRaw
start-A
start-b
A-c
A-b
b-d
A-end
b-end

In [ ]:
#!value --name inputRaw
dr-of
start-KT
yj-sk
start-gb
of-start
IJ-end
VT-sk
end-sk
VT-km
KT-end
IJ-of
dr-IJ
yj-IJ
KT-yj
gb-VT
dr-yj
VT-of
PZ-dr
KT-of
KT-gb
of-gb
dr-sk
dr-VT

In [ ]:
type Node = 
    | Start
    | End
    | Large of string
    | Small of string
type Path = Node list

In [ ]:
module Formatting =
    let formatNode node =
        match node with
        | Start -> "START"
        | End -> "END"
        | Large x -> x.ToUpperInvariant()
        | Small x -> x.ToLowerInvariant()
    let displayNode = formatNode >> display
    let formatNodePair (n1, n2) = 
        $"{formatNode n1} → {formatNode n2}"
    let displayNodePair = formatNodePair >> display
    let formatPath list = 
        String.Join(" → ", (list |> List.map (formatNode)))
    let displayPath = formatPath >> display

In [ ]:
#load "../common.fsx"

let parse input =
    let node s =
        match s with
        | "start" -> Start
        | "end" -> End
        | x when x.ToUpperInvariant() = x -> Large x
        | x when x.ToLowerInvariant() = x -> Small x
        | _ -> failwith "Unexpected input"
    readLines input 
    |> Seq.map (splitToTuple2 [|"-"|]) 
    |> Seq.map (fun (n1, n2) -> node n1, node n2)
    |> Seq.collect (fun (x1,x2) -> [|x1,x2; x2,x1|])
    |> Array.ofSeq

#!share sampleRaw --from value
#!share inputRaw --from value
let sampleAdjacency = parse sampleRaw 
let actualAdjacency = parse inputRaw

"All adjacencies:" |> display
sampleAdjacency |> Array.map Formatting.formatNodePair |> display

All adjacencies:

index,value
0,START → A
1,A → START
2,START → b
3,b → START
4,A → c
5,c → A
6,A → b
7,b → A
8,b → d
9,d → b


In [ ]:
let rec findPaths currentNode toNode adjacencies (visited: Map<Node, int>) visitCriteria =
    let newVisited = Map.change currentNode (fun x -> Some ((defaultArg x 0) + 1)) visited
    if (currentNode = toNode)
    then [[toNode]]
    else
        let whereCanGo = 
            adjacencies
            |> Array.where (fst >> ((=)currentNode)) |> Array.map snd
            |> Array.filter (visitCriteria newVisited)
        whereCanGo
        |> Array.map (fun candidate -> findPaths candidate toNode adjacencies newVisited visitCriteria)
        |> List.ofArray
        |> List.collect id
        |> List.map (fun x -> currentNode::x)
        |> List.distinct

In [ ]:
let part1CanVisitCriteria visited candidate = 
    match candidate with
    | Start -> not <| Map.containsKey Start visited
    | End -> not <| Map.containsKey End visited
    | Large _ -> true
    | Small x -> not <| Map.containsKey (Small x) visited
    

In [ ]:
findPaths Start End sampleAdjacency Map.empty part1CanVisitCriteria
|> List.map Formatting.formatPath

index,value
0,START → A → c → A → b → A → END
1,START → A → c → A → b → END
2,START → A → c → A → END
3,START → A → b → A → c → A → END
4,START → A → b → A → END
5,START → A → b → END
6,START → A → END
7,START → b → A → c → A → END
8,START → b → A → END
9,START → b → END


In [ ]:
findPaths Start End actualAdjacency Map.empty part1CanVisitCriteria 
|> List.length

4691

In [ ]:
let part2CanVisitCriteria visited candidate =
    match candidate with
    | Start -> not <| Map.containsKey Start visited
    | End -> not <| Map.containsKey End visited
    | Large _ -> true
    | Small x -> not <| Map.containsKey (Small x) visited
                || visited
                   |> Map.toSeq
                   |> Seq.choose (fun (node, visitCount) -> match node with | Small y -> Some visitCount | _ -> None)
                   |> Seq.forall (fun smallVisits -> smallVisits < 2)


In [ ]:
Formatter.ListExpansionLimit <- 50

findPaths Start End sampleAdjacency Map.empty part2CanVisitCriteria
|> List.map Formatting.formatPath

index,value
0,START → A → c → A → c → A → b → A → END
1,START → A → c → A → c → A → b → END
2,START → A → c → A → c → A → END
3,START → A → c → A → b → A → c → A → END
4,START → A → c → A → b → A → b → A → END
5,START → A → c → A → b → A → b → END
6,START → A → c → A → b → A → END
7,START → A → c → A → b → d → b → A → END
8,START → A → c → A → b → d → b → END
9,START → A → c → A → b → END


In [ ]:
findPaths Start End actualAdjacency Map.empty part2CanVisitCriteria 
|> List.length

140718